In [1]:
import pandas as pd

In [3]:
# read complex Excel file
# skip the first 10 rows, read the next 5 rows, and use the columns from 'B' to 'D'
df = pd.read_excel(
    r"excel_table.xlsx",
    sheet_name="Sheet1",
    skiprows=range(1, 2),
    nrows=5,
    usecols="B:D",
)
df

,A,B,C
0,1.606687,-0.592937,-0.333606
1,-1.306918,2.504540,-0.915892
2,-1.645197,-0.655435,-0.451916
3,-1.106023,0.998995,0.051905
4,0.127527,-1.072797,0.612099


In [5]:
# write 2 df to 2 sheets in same Excel file
df2 = df.assign(B=df["B"] * 2).rename(columns={"B": "B2"})

with pd.ExcelWriter("output.xlsx") as writer:
    df.to_excel(writer, sheet_name="Sheet1", index=False)
    df2.to_excel(writer, sheet_name="Sheet2", index=False)

In [ ]:
import pandas as pd

# Example DataFrame for table1
table1 = pd.DataFrame({
    "ID": [1, 2, 3],
    "Name": ["Alice", "Bob", "Charlie"],
    "Amount": [100, 200, 150]
})

# Example DataFrame for table2
table2 = pd.DataFrame({
    "OrderID": [101, 102],
    "Product": ["Widget", "Gadget"],
    "Quantity": [5, 10]
})

# Example DataFrame for table3
table3 = pd.DataFrame({
    "OrderID": [201, 202],
    "Status": ["Shipped", "Pending"],
    "Date": ["2025-06-01", "2025-06-02"]
})

# Example DataFrame for table4
table4 = pd.DataFrame({
    "LoanID": [301, 302, 303],
    "Customer": ["Dave", "Eve", "Frank"],
    "Balance": [1000, 1500, 1200]
})

# Example output path
out_excel_path = "example_output.xlsx"
# ...existing code...
from pandas import ExcelWriter
from openpyxl import Workbook
from openpyxl.styles import Border, Side

with ExcelWriter(out_excel_path, engine="openpyxl", mode="w") as writer:
    table1.to_excel(writer, sheet_name="table1", index=False)
    table2.to_excel(writer, sheet_name="table2", index=False)
    table3.to_excel(writer, sheet_name="table3", index=False)
    table4.to_excel(writer, sheet_name="table4", index=False)

    # Remove header borders for all sheets
    workbook = writer.book
    for sheet_name in ["table1", "table2", "table3", "table4"]:
        ws = workbook[sheet_name]
        for cell in ws[1]:  # First row (header)
            cell.border = Border(left=Side(border_style=None),
                                right=Side(border_style=None),
                                top=Side(border_style=None),
                                bottom=Side(border_style=None))
# ...existing code...